In [ ]:
!nvidia-smi

In [2]:
!pip3 install datasets
!pip3 install rouge_score
!pip3 install git+https://github.com/huggingface/transformers
!pip3 install sentencepiece

     |################################| 234 kB 18.3 MB/s eta 0:00:01
     |################################| 20.7 MB 18.1 MB/s eta 0:00:01
     |################################| 81 kB 14.0 MB/s eta 0:00:01
     |################################| 111 kB 30.8 MB/s eta 0:00:01
     |################################| 101 kB 15.6 MB/s ta 0:00:01
     |################################| 69 kB 14.3 MB/s eta 0:00:01
     |################################| 9.5 MB 17.7 MB/s eta 0:00:01
     |################################| 243 kB 28.4 MB/s eta 0:00:01
     |################################| 510 kB 26.3 MB/s eta 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.51.0
    Uninstalling tqdm-4.51.0:
      Successfully uninstalled tqdm-4.51.0
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |################################| 1.5 MB 35.5 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install

In [16]:
DATASET_NAME = "pubmed"
device = "cuda"

MODEL_ID = f"google/bigbird-pegasus-large-{DATASET_NAME}"
from transformers import AutoTokenizer, AutoModelForMaskedLM
  

from transformers import EncoderDecoderModel, BigBirdModel, BigBirdForCausalLM, BigBirdTokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
#from transformers import PegasusTokenizer, BigBirdPegasusModel
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
import torch
model = BigBirdPegasusForConditionalGeneration.from_pretrained(MODEL_ID, gradient_checkpointing=True, use_cache=False).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed",use_fast=False)
#tokenizer = AutoTokenizer.from_pretrained("deepset/covid_bert_base")

In [1]:
from datasets import load_dataset
import pandas as pd
train=pd.read_csv('trainer.csv')

In [2]:
train=train[['abstract','articles', 'sections']]
train.rename(columns = {'sections':'section_names'}, inplace = True)

In [5]:
train.shape

(10563, 3)

In [4]:
test=train[1:100]
val=train[100:300]
train=train[300:10863]

In [6]:
import re
def finalclean(c):
  c = re.sub(r'[^A-Za-z0-9\n., %]+', " ",c)
  c=re.sub(r'Table [0-9]*',"", c)
  return c

def removeurls(text):
    result = re.sub(r"http\S+", "", text)
    result = re.sub(r"www\S+", "", result)
    return result
def removbrackets(text):
    text=re.sub("[\(\[].*?[\)\]]", "", text)
    return text
def misc_clean(text):
    d=re.sub('[a-z0-9.-]*/[a-z0-9.-]*'," ", text)
    return d

def single(text):
   # Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    return text
def removelines(text):
   # Single character removal
    text = re.sub(r"\n", ' ', text)
    return text

In [9]:
train['articles'] = train['articles'].apply(lambda x: finalclean(x))
val['articles'] = val['articles'].apply(lambda x: finalclean(x))

train['articles'] = train['articles'].apply(lambda x: removeurls(x))
val['articles'] = val['articles'].apply(lambda x: removeurls(x))
train['abstract'] = train['abstract'].apply(lambda x: removeurls(x))
val['abstract'] = val['abstract'].apply(lambda x: removeurls(x))

train['articles'] = train['articles'].apply(lambda x: removbrackets(x))
val['articles'] = val['articles'].apply(lambda x: removbrackets(x))
train['abstract'] = train['abstract'].apply(lambda x: removbrackets(x))
val['abstract'] = val['abstract'].apply(lambda x: removbrackets(x))

train['abstract'] = train['abstract'].apply(lambda x: misc_clean(x))
val['abstract'] = val['abstract'].apply(lambda x: misc_clean(x))
train['articles'] = train['articles'].apply(lambda x: misc_clean(x))
val['articles'] = val['articles'].apply(lambda x: misc_clean(x))

train['abstract'] = train['abstract'].apply(lambda x: finalclean(x))
val['abstract'] = val['abstract'].apply(lambda x: finalclean(x))

train['articles'] = train['articles'].apply(lambda x: single(x))
val['articles'] = val['articles'].apply(lambda x: single(x))

train['articles'] = train['articles'].apply(lambda x: removelines(x))
val['articles'] = val['articles'].apply(lambda x: removelines(x))
train['abstract'] = train['abstract'].apply(lambda x: removelines(x))
val['abstract'] = val['abstract'].apply(lambda x: removelines(x))

In [11]:
print(train.isnull().sum(), val.isnull().sum())

abstract         0
articles         0
section_names    0
dtype: int64 abstract         0
articles         0
section_names    0
dtype: int64


In [12]:
from datasets import Dataset
train = Dataset.from_pandas(train)
val = Dataset.from_pandas(val)

In [13]:
train, val

(Dataset({
     features: ['abstract', 'articles', 'section_names'],
     num_rows: 10563
 }),
 Dataset({
     features: ['abstract', 'articles', 'section_names'],
     num_rows: 200
 }))

In [14]:
def collate_fn(features):
    

    article = [x["articles"] for x in features]
    section_names = [x["section_names"] for x in features]
    abstract = [x["abstract"] for x in features]

  # should not eliminate special tokens since question and context are should have `SEP` in middle
    inputs = tokenizer(article, section_names , return_tensors="pt", padding="max_length", truncation=True, max_length=4096)
    labels = tokenizer(abstract, return_tensors="pt", padding=True, truncation=True, max_length=256)

    return {
      "input_ids": inputs.input_ids,
      "attention_mask": inputs.attention_mask,
      #"decoder_input_ids": labels.input_ids,
      "labels": labels.input_ids,
      "decoder_attention_mask": labels.attention_mask,
  }


In [21]:
 args = Seq2SeqTrainingArguments(
    output_dir="summary",
    overwrite_output_dir=False,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    # eval_steps=4000,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    num_train_epochs=5,
    logging_strategy="epoch",
    logging_steps=4000,
    save_strategy="epoch",
    run_name="BBsummary",
    disable_tqdm=False,
    load_best_model_at_end=True,
    remove_unused_columns=False
    #fp16=True,
   
)

trainer = Seq2SeqTrainer(
      model=model,
      args=args,
      data_collator=collate_fn,
      train_dataset=train,
      eval_dataset=val,
  )

In [22]:
trainer.train('summary/checkpoint-7923')

Epoch,Training Loss,Validation Loss
4,2.321700,2.275265
5,2.248500,2.262482


TrainOutput(global_step=13205, training_loss=0.9140246355547141, metrics={'train_runtime': 66758.2335, 'train_samples_per_second': 0.791, 'train_steps_per_second': 0.198, 'total_flos': 8.691069077201879e+17, 'epoch': 5.0})

In [30]:
import os
print(os.getcwd())

/notebooks


In [23]:
trainer.save_model("BigBirdTrained_CORD")

In [39]:
def generate_answer(batch):
    inputs_dict = tokenizer(batch["articles"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    inputs_dict = {k: inputs_dict[k].to(device) for k in inputs_dict}
    predicted_abstract_ids = model1.generate(**inputs_dict, max_length=256, num_beams=5, length_penalty=0.8)
    batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True)
    print(batch["predicted_abstract"])
    return batch

In [35]:
from datasets import load_dataset, load_metric
rouge = load_metric("rouge")

In [40]:
dataset_small = train.select(range(5))
result_small = dataset_small.map(generate_answer)

rouge.compute(predictions=result_small["predicted_abstract"], references=result_small["abstract"])

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
model1=BigBirdPegasusForConditionalGeneration.from_pretrained("final-model_epoch3", gradient_checkpointing=True, use_cache=False).to(device)



In [41]:
!nvidia_smi

/bin/sh: 1: nvidia_smi: not found
